In [1]:
import json
import numpy as np
from tqdm import tqdm
import json
import os

# Step 2: Add the package directory to Python's sys.path
import sys
solution_name = 'solution'
sys.path.append(f'/kaggle/input/{solution_name}/')

# Step 3: Import your custom package or module

In [2]:
from abstract_and_reason import solver_v1

In [3]:
base_path = '/kaggle/input/arc-prize-2024/'

# Loading JSON data
def load_json(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

# Reading files
test_challenges = load_json(base_path +'arc-agi_test_challenges.json')
submission_file_path = '/kaggle/working/submission.json'
sample_submission_file_path = base_path + 'sample_submission.json'
sample_submission = load_json(sample_submission_file_path)

In [4]:
# Ensure the submission file exists or create an empty one
if os.path.exists(sample_submission_file_path):
    sample_submission = load_json(sample_submission_file_path)
else:
    sample_submission = {}

if not os.path.exists(submission_file_path):
    with open(submission_file_path, "w") as file:
        json.dump(sample_submission, file, indent=4)

In [5]:
# Initialize solver
anr = solver_v1.Solver(prod=True)

In [6]:
# Open the submission file for reading and updating
with open(submission_file_path, "r+") as outfile:
    # Load existing data from submission.json
    submission_data = json.load(outfile)
    
    ids_test = list(test_challenges)

    # Process each challenge and update submission_data
    for i, challenge_id in enumerate(tqdm(ids_test)):
        puzzle_ins_train, puzzle_outs_train, puzzle_ins_test, puzzle_outs_test = anr.process_challenge(challenge_id, test_challenges)
        
        # Predict the attempts
        attempt1 = anr.predict(puzzle_ins_train, puzzle_outs_train, puzzle_ins_test)
        attempt2 = anr.predict(puzzle_ins_train, puzzle_outs_train, puzzle_ins_test)
        
        # Prepare the data to be written
        result = []
        for j in range(len(attempt1)):
            result.append({
                'attempt_1': attempt1[j].tolist(),
                'attempt_2': attempt2[j].tolist()
            })
        
        # Update submission_data in memory
        submission_data[challenge_id] = result
        
        # Write updated submission_data back to the file
        outfile.seek(0)  # Go to the beginning of the file
        json.dump(submission_data, outfile, indent=4)
        outfile.truncate()  # Remove any leftover data after the current position

100%|██████████| 100/100 [00:02<00:00, 43.14it/s]
